To save in google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


**Packages**

In [3]:
! pip install datasets --quiet
! pip install evaluate --quiet
! pip install rouge_score --quiet
! pip install sacrebleu --quiet
! pip install transformers --quiet
! pip install -q sentencepiece --quiet
! pip install summarizer --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.4 MB/s eta 

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from datasets import load_dataset
import evaluate

from pprint import pprint

**Data**

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/W266FinalProject/Datasets/xl_sum_sample_train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/W266FinalProject/Datasets/xl_sum_sample_val.csv')
test_df = pd.read_csv('/content/drive/MyDrive/W266FinalProject/Datasets/xl_sum_sample_test.csv')

In [6]:
print(f'train shape: {train_df.shape}')
print(f'val size: {val_df.shape}')
print(f'test size: {test_df.shape}')

train shape: (1000, 2)
val size: (100, 2)
test size: (100, 2)


In [7]:
train_df.head(n=2)

,text,summary
0,By Rebecca Ricks & Johnny O'SheaBBC Spotlight ...,"During the spring, at the height of the Covid-..."
1,"By Rachel SchraerBBC Reality Check So, why did...","The parents of five-year-old Tafida Raqeeb, wh..."


In [8]:
def get_length(text):
    return len(text)

article_ave = train_df['text'].apply(get_length).mean()
article_max = train_df['text'].apply(get_length).max()
article_min = train_df['text'].apply(get_length).min()
article_std = train_df['text'].apply(get_length).std()

print(f'average article size: {article_ave}')
print(f'max article size: {article_max}')
print(f'min article size: {article_min}')
print(f'stdev article size: {article_std}')

average article size: 2685.591
max article size: 14526
min article size: 300
stdev article size: 1815.5251965914597


In [9]:
summ_ave = train_df['summary'].apply(get_length).mean()
summ_max = train_df['summary'].apply(get_length).max()
summ_min = train_df['summary'].apply(get_length).min()
summ_std = train_df['summary'].apply(get_length).std()

print(f'average summary size: {summ_ave}')
print(f'max summary size: {summ_max}')
print(f'min summary size: {summ_min}')
print(f'stdev summary size: {summ_std}')

average summary size: 129.662
max summary size: 497
min summary size: 45
stdev summary size: 40.11242653795408


###### Prepare data for finetuning on summarize task

In [10]:
train_df['text'] = train_df['text'].apply(lambda x: 'summarize: '+x)
val_df['text'] = val_df['text'].apply(lambda x: 'summarize: '+x)
test_df['text'] = test_df['text'].apply(lambda x: 'summarize: '+x)

**T5 Model**

##### 1.) Load and set up model

In [11]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [12]:
t5model.config

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "pre

In [13]:
## Set up model params

model_name = 't5_finetuned8'
source_max_length = 512
target_max_length = 128
batch_size = 32

In [17]:
def preprocess_data(text_pairs, tokenizer, model, source_max_length=512, target_max_length=128):
    orig_text = text_pairs[0]
    orig_encoded = tokenizer.batch_encode_plus(
        orig_text,
        max_length=source_max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

    orig_input_ids = np.array(orig_encoded["input_ids"], dtype="int32")
    orig_attention_masks = np.array(orig_encoded["attention_mask"], dtype="int32")
    
    target_text = text_pairs[0]
    target_encoded = tokenizer.batch_encode_plus(
        target_text,
        max_length=target_max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

    label_ids = np.array(target_encoded['input_ids'])
    decoder_input_ids = model._shift_right(label_ids)
    
    return [orig_input_ids, orig_attention_masks, decoder_input_ids], label_ids

In [18]:
import tensorflow as tf

class SummarizationDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self,
                 tokenizer,
                 model,
                 n_examples,
                 dataframe,
                 source_max_length=128,
                 target_max_length=64,
                 batch_size=16,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.model = model
        self.n_examples = n_examples
        self.dataframe = dataframe
        self.source_max_length = source_max_length
        self.target_max_length = target_max_length
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        # Return the number of batches in the full dataset
        return self.n_examples // self.batch_size
    
    def __getitem__(self, idx):
        batch_start = idx * self.batch_size
        batch_end = (idx + 1) * self.batch_size

        # Indices to skip are the ones in the shuffled row_order before and
        # after the chunk we'll use for this batch
        batch_idx_skip = self.row_order[:batch_start] + self.row_order[batch_end:]
        
        text_pairs = self.dataframe[['text', 'summary']].values.astype(str).tolist()
        
        batch_data = preprocess_data(
            text_pairs,
            self.tokenizer,
            self.model,
            self.source_max_length,
            self.target_max_length
        )

        return batch_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [19]:
from tensorflow.keras import layers

def build_t5_training_wrapper_model(t5_model, source_max_length, target_max_length):
    input_ids = layers.Input(shape=(source_max_length), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(source_max_length), dtype=tf.int32, name='attention_mask')
    decoder_input_ids = layers.Input(shape=(target_max_length), dtype=tf.int32, name='labels')
    
    t5_logits = t5_model(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)[0]

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask, decoder_input_ids],
                                  outputs=[t5_logits])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model

##### 2.) Train model

In [20]:
model_wrapper = build_t5_training_wrapper_model(t5model, source_max_length, target_max_length)

In [21]:
train_data_generator = SummarizationDataGenerator(
    tokenizer=t5tokenizer,
    model=t5model,
    n_examples=train_df.shape[0],
    dataframe=train_df,
    source_max_length=source_max_length,
    target_max_length=target_max_length,
    batch_size=batch_size
)

valid_data_generator = SummarizationDataGenerator(
    tokenizer=t5tokenizer,
    model=t5model,
    n_examples=val_df.shape[0],
    dataframe=val_df,
    source_max_length=source_max_length,
    target_max_length=target_max_length,
    batch_size=batch_size
)

In [22]:
checkpoint_dir = f'/content/drive/MyDrive/W266FinalProject/model_checkpoints/{model_name}/'
checkpoint_filepath = checkpoint_dir + 't5_weights.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True)

In [23]:
model_wrapper.fit(train_data_generator,
                  validation_data=valid_data_generator,
                  epochs=5,
                  callbacks=[model_checkpoint_callback])

Epoch 1/5
31/31 [==============================] - 1035s 31s/step - loss: 0.0416 - accuracy: 0.9924 - val_loss: 0.1169 - val_accuracy: 0.9766
Epoch 2/5
31/31 [==============================] - 925s 30s/step - loss: 0.0097 - accuracy: 0.9977 - val_loss: 0.1442 - val_accuracy: 0.9531
Epoch 3/5
31/31 [==============================] - 906s 29s/step - loss: 0.0081 - accuracy: 0.9985 - val_loss: 0.2245 - val_accuracy: 0.9609
Epoch 4/5
31/31 [==============================] - 906s 29s/step - loss: 0.0150 - accuracy: 0.9974 - val_loss: 0.1278 - val_accuracy: 0.9727
Epoch 5/5
31/31 [==============================] - 898s 29s/step - loss: 0.0040 - accuracy: 0.9992 - val_loss: 0.1993 - val_accuracy: 0.9453


##### 3.) Test model

In [14]:
rouge = evaluate.load('rouge')

In [15]:
chrf = evaluate.load("chrf")

In [20]:
model_wrapper = build_t5_training_wrapper_model(t5model, source_max_length, target_max_length)

In [21]:
checkpoint_filepath = f'/content/drive/MyDrive/W266FinalProject/model_checkpoints/{model_name}/t5_weights.04-0.97.hdf5'

model_wrapper.load_weights(checkpoint_filepath)

In [22]:
test_df.shape

(100, 2)

In [23]:
r1 = []
r2 = []
rL = []
rLs = []
chrfs = []

for i in test_df.index:

    T5ARTICLE_TO_SUMMARIZE = test_df['text'][i]

    inputs = t5tokenizer(T5ARTICLE_TO_SUMMARIZE, 
                         max_length=source_max_length, 
                         truncation=True, 
                         return_tensors="tf")

    summary_ids = t5model.generate(inputs["input_ids"], max_length=target_max_length)

    candidate = t5tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    #pprint(candidate[0], compact=True)

    ref = [test_df['summary'][i]]

    rouge_results = rouge.compute(predictions=candidate,
                                  references=ref)

    r1.append(rouge_results['rouge1'])
    r2.append(rouge_results['rouge2'])
    rL.append(rouge_results['rougeL'])
    rLs.append(rouge_results['rougeLsum'])
    
    chrf_results = chrf.compute(predictions=candidate,
                                references=ref)
    chrfs.append(chrf_results['score'])

In [24]:
print('rouge1 average :', np.mean(r1))
print('rouge2 average :', np.mean(r2))
print('rougeL average :', np.mean(rL))
print('rougeLs average :', np.mean(rLs))
print('chrf average :', np.mean(chrfs))

rouge1 average : 0.15352663117632165
rouge2 average : 0.024369720389626925
rougeL average : 0.11044849952248244
rougeLs average : 0.11044849952248244
chrf average : 23.638883878151788


In [25]:
data = {'rouge1': r1, 'rouge2': r2, 'rogueL': rL, 'rogueLs': rLs, 'chrf': chrfs}

scores = pd.DataFrame(data)

scores.to_csv(f'/content/drive/MyDrive/W266FinalProject/model_results/{model_name}_scores.csv', index=False)